<b>Import necessery libraries</b>


In [315]:
#!pip install xlwt

In [316]:
import spacy
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from azureml.core import Experiment
from azureml.core import Workspace, Dataset
from azureml.data import DataType
from spacy.cli.download import download as spacy_download
import os 
from os.path import join as osjoin
import xlwt

In [280]:
#!pip install --upgrade spacy

<b>Select the default workspace & datastore</b>

In [281]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc'
resource_group = 'MLRG'
workspace_name = 'erbbimlws'

workspace = Workspace(subscription_id, resource_group, workspace_name)


datastore = workspace.get_default_datastore()



<b>Loading the Greek language tools</b>

In [282]:
spacy_download('el_core_news_sm')
nlp =spacy.load('el_core_news_sm', disable=['tagger', 'parser', 'ner'])

✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


<b>Parameter definitions</b>

In [283]:
#minimum number of tokens in the texts
minCount = 10
#ngrams parameters
mindf,minngram,maxngram = 30,2,3
#keep empty tokens
deleteEmptyTokens = True
#dataset name to be analyzed
datasetName = 'Txns_NLP_202011_part2'
#export filename
fileName = 'Txns_NLP_202011_part2_exp'

<b>Regular expressions definitions</b>

In [284]:
p1 = re.compile('δεν απαντ.{1,3}\s{0,1}',re.IGNORECASE)
p2 = re.compile('\sδα\s',re.IGNORECASE)
p3 = re.compile('δε.{0,1}\s.{0,3}\s{0,1}βρ.{1,2}κ.\s{0,1}',re.IGNORECASE)
p4 = re.compile('[^\d]?\d{10}')
p5 = re.compile('[^\d]?\d{18}|[^\d]\d{20}')
p6 = re.compile('δε[ ν]{0,1} (επιθυμ[α-ω]{2,4}?|[ήη]θ[εέ]λ[α-ω]{1,3}?|θελ[α-ω]{1,4}|.{0,20}ενδιαφ[εέ]ρ[α-ω]{2,4})',re.IGNORECASE)
p7 = re.compile('δε[ ν]{0,1} (μπορ[α-ω]{2,5}|.εχει)',re.IGNORECASE)
p8 = re.compile('(δεν|μη).*διαθεσιμ[οη]ς{0,1}?',re.IGNORECASE)
p9 = re.compile('(δεν|μη)+.*εφικτη?',re.IGNORECASE)
p10 = re.compile('δε[ ν]{0,1}.{0,20}θετικ[οόήη]ς{0,1}',re.IGNORECASE)

#pinakides
#p11 = re.compile('\s([a-zA-Zα-ωΑ-Ω]{3}\s*?[0-9]{3,4})\s',re.IGNORECASE)
p11 = re.compile('\s[a-zA-Zα-ωΑ-Ω]{3}[0-9]{3,4}\s',re.IGNORECASE)

#enoikia
p12=re.compile('εν(.{1,2}κ.{1,3})',re.IGNORECASE)
p13=re.compile('en(.{1,2}k.{1,3})',re.IGNORECASE)

p14 = re.compile('(φυσιοθεραπ|ορθοπαιδικος|ΩΡΛ|ΩΝΑΣΕΙΟ|ψυχολογος|ψυχοθεραπεια|ψυχιατρος)\S*' ,re.IGNORECASE)
p15 = re.compile('(EX/SH|Εξοφληση|ΕΞΟΦΛΗΣΗ|eksoflisi|ΕΞΩΦΛΗΣΗ|Εξόφλησ)\S*' ,re.IGNORECASE)
p16 = re.compile('(ASFALIA|asfaleia|ασφάλ|asfalistra)\S*' ,re.IGNORECASE)
p17 = re.compile('(ΤΙΜΟΛΟΓΙΟ|ΤΙΜΟΛ|ΤΙΜΟΛΟΓΊΟΥ|TIM|ΠΡΟΦΟΡΜΑ)\S*' ,re.IGNORECASE)
p18 = re.compile('(ΝΕΡΟΥ|ΝΕΡΟ|ευδαπ|ΔΕΗ|dei|ρευμα|nero|nerou|eydap)\S*' ,re.IGNORECASE)
p19 = re.compile('(ΜΙΣΘΟΔΟΣΙΑ|Αδεια|ΑΠΟΔΟΧΕΣ|μηνιατικο|μισθοδοσία|ΕΞΟΦΛ ΜΙΣΘ|ΜΙΣΘΟΣ|ΜΙΣΘΟΔΩΣΙΑ|ΜΙΣΘ/ΣΙΑ|ΔΩΡΟ ΧΡΙΣΟΥΓΕΝΩΝ|ΔΩΡΟ ΧΡΙΣΤ|ΟΔΟΙΠΟΡΙΚΑ)\S*' ,re.IGNORECASE)
p20 = re.compile('(ΕΝΦΙΑ|enfia|ΤΑΠ ΑΚΙΝΗΤΟΥ|ΚΤΗΜΑΤΟΛΟΓΙΟ)\S*' ,re.IGNORECASE)
p21 = re.compile('(ΕΦΚΑ|efka|foros|φορος|φορος εισοδηματος|ΦΠΑ)\S*' ,re.IGNORECASE)
p22 = re.compile('(KOINOXΡHΣTA|ΚΟΙΝ/ΣΤΑ|Συντηρηση Ανελκυστηρα|ΚΟΙΝ|Κοινόχροιστα|KOINOXRISTA)\S*' ,re.IGNORECASE)
p23 = re.compile('(να ζησετε|δωρο γαμου|na zisete|gamou|gamos|γαμος)\S*' ,re.IGNORECASE)
p24 = re.compile('(να σας ζησει|na sas zisei|βαφτιση|vaftisi|baftisi)\S*' ,re.IGNORECASE)
p25 = re.compile('(δοση σπίτι|Δάνειο σπίτι|δόση στεγαστικού δανείου|ΣΤΕΓΑΣΤΙΚΟ  ΔΑΝΕΙΟ|Πληρωμη στεγαστικου)\S*' ,re.IGNORECASE)
p26 = re.compile('(μτφ|metafora|μεταφορα|mtf)\S*' ,re.IGNORECASE)
p27 = re.compile('(ORDER|παραγγελίας|ΠΑΡΑΓΓΕΛΙΑ|αρ.παρ.|PARAGGELIAS|PARAGGELIA|αρ.παρ|orders)\S*' ,re.IGNORECASE)
p28 = re.compile('(xreos|ΔΑΝΕΙΚΑ|ΕΝΑΝΤΙ ΛΟΓΑΡΙΑΣΜΟΥ|εξοδα|ΕΞΟΔ.|expenses|οφειλη|χρεος)\S*' ,re.IGNORECASE)
p29 = re.compile('(κτθ|καταθεση|katathesi|ktth)\S*' ,re.IGNORECASE)
p30 = re.compile('(ΠΡΟΚΑΤΑ|ΠΡΟΚΑΤΑΒΟΛΉ|ΠΡΟΚΑΤΑΒ|prokatavoli|prokataboli|prokatav)\S*' ,re.IGNORECASE)
p31 = re.compile('(DIATROFI|ΔΙΑΤΡΟΦΗ|ΔΙΑΤΡΟΦΙ)\S*' ,re.IGNORECASE)
p32 = re.compile('(δοση αυτοκινητου|δανειο αυτοκινητου|daneiou autokinitou|daneio autokinitou|dosi autokonitou|daneiou autokinito)\S*' ,re.IGNORECASE)
p33 = re.compile('(ΠΛΗΡΩΜΗ ΔΑΝΕΙΟΥ|dosi daniou|εξοικονομω|Πληρωμη δοσης|δοση δανειου|ΔΟΣΗ ΔΑΝΕΙΟΥ|DOSΗ ΔΑΝΕΙΟΥ|ΔΌΣΗΔΑΝΕΊΟΥ|ΠΛΥΡΟΜΗ.ΔΑΝΕΙΟΥ)\S*' ,re.IGNORECASE)
p34 = re.compile('(Πληρωμή καρτας|ΠΛΗΡΩΜΉ ΠΙΣΤΩΤΙΚΗΣ)\S*' ,re.IGNORECASE)




<b>Functions definitions</b>

In [285]:
def loadStopWords(ws):
    #A dataset containing the Greek stop words has been created
    #the function loads this dataset as a dataframe
    dataset = Dataset.get_by_name(ws, name='stopWords_gr')
    sw = set(dataset.to_pandas_dataframe())
    return sw

In [286]:
def replaceTerm(text):
    #This function uses the above defined regular expressions to replace text
    #The order of the rules is importand
    #Compinations of two or more words, are concatenated, in order to be considered as a single token
    
    #text = str(text)
    
    text = p5.sub(' λογαριασμός ',text)
    text = p4.sub(' τηλεφωνο ',text)
    text = p6.sub(' δενθελειδενενδιαφερεται ',text)
    text = p10.sub(' δενθελειδενενδιαφερεται ',text)
    text = p7.sub(' δενεχειδενμπορει ',text)
    text = p8.sub(' δενειναιδιαθεσιμος ',text)
    text = p9.sub(' ανεφικτη ',text)
    text = text.replace('-banking','banking')
    text = text.replace('v banking','vbanking')
    text = text.replace('e banking','ebanking')
    text = text.replace('follow up','followup')
    text = text.replace('fup','followup')
    text = text.replace('f/up','followup')
    text = text.replace('πυρ/ριο','πυρασφαλιστηριο')
    text = text.replace('safe drive','safedrive')
    text = text.replace('safe pocket','safepocket')
    text = text.replace('alphabank','alpha')
    text = text.replace('sweet home smart','sweethomesmart')
    text = text.replace('sweet home','sweethome')
    text = text.replace('eξασφαλιζω','εξασφαλιζω')
    text = text.replace('credit card','creditcard')
    text = text.replace('debit card','debitcard')
    text = text.replace('life cycle','lifecycle')
    text = text.replace('π/κ','πκ')
    text = text.replace('td','πκ')
    text = text.replace('α/κ','ακ')
    text = text.replace('δ/α','δεναπαντα ')
    text = text.replace('εκτος αττικης','εκτοςαττικης ')
    text = text.replace('PaF online payments','πληρωμήPaF')
    text = text.replace('ΚΤΘ APS', 'κατάθεσηAPS')
    text = text.replace('ΝΟΜΟΣ 3869', 'ΝΟΜΟΣ3869')
    text = text.replace('ΜΤΦ ΜΕΣΩ WEB','ΣυστημικόΜΤΦebanking')
    text = text.replace('VISA GOLD', 'VISAGOLD')
    text = text.replace('Ασφαλεια αυτο/του','ασφαλειααυτοκινήτου')
    text = text.replace('ασφαλεια αυτοκινητου','ασφαλειααυτοκινήτου')
    text = text.replace('ασφαλειων αυτ/των','ασφαλειααυτοκινήτου')
    text = text.replace('ΑΣΦΆΛΕΙΑ ΣΠΙΤΙΟΎ','ασφαλειασπιτιου')
    text = text.replace('POI',' εισερχομενοεμβασμα ')
    text = text.replace('POO',' εξερχομενοεμβασμα ')
    
    
    #τδ
    text = p1.sub(' δεναπαντα ',text)
    text = p2.sub(' δεναπαντα ',text)
    text = p3.sub(' δεντονβρηκα ',text)
    text = p17.sub(' τιμολογια ',text)
    text = p11.sub(' αριθμοςκυκλοφοριας ', text)
    text = p12.sub(' ενοικιο ',text)
    text = p13.sub(' ενοικιο ',text)
    text = p14.sub(' γιατροί ',text)
    text = p15.sub(' εξόφληση ',text)
    text = p16.sub(' ασφάλειες ',text)
    
    text = p18.sub(' ΔΕΚΟ ',text)
    text = p19.sub(' μισθοδοσιες ', text)
    text = p20.sub(' εφοριαακινητων ', text)
    text = p21.sub(' εφορια ', text)
    text = p22.sub(' κοινοχρηστα ', text)
    text = p23.sub(' συναλλαγεςγαμου ', text)
    text = p24.sub(' συναλλαγεςγεννησηςβαφτισης ', text)
    text = p25.sub(' στεγαστικοδανειο ', text)
    text = p26.sub(' μεταφορες ', text)
    text = p27.sub(' παραγγελίες ', text)
    text = p28.sub(' οφειλες ', text)
    text = p29.sub(' καταθεσεις ', text)
    text = p30.sub(' προκαταβολες ', text)
    text = p31.sub(' διατροφη ', text)
    text = p32.sub(' δανειοαυτοκινητου ', text)
    text = p33.sub(' πληρωμηδανειου ', text)
    text = p34.sub(' πληρωμηπιστωτικης ', text)
    
    
    return text

In [287]:
def remove_ton(text):
    #removes punctuation, αφαιρεί τους τόνους
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   

In [288]:
def clean_text(text,sw):
    #This function performs text cleansing and returns the clean and lemmatized version of the original text
    #conver to lower text 
    text = str(text).lower()
    
   # remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]

    # αφαιρούνται οι τόνοι
    text = [remove_ton(x) for x in text]
    
    # remove stop words
    text = [x for x in text if x not in sw]
    
    #replacements either by rules or regular expressions
    
    
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in {'quot','amp'}]
    
    # remove words that contain numbers
    #text = [word for word in text if not any(c.isdigit() for c in word)] #addition to return even empty tokens
    
    # remove empty tokens
    text = [t for t in text if len(t) > 0] #addition to return even empty tokens
    
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 0]) #addition to return even empty tokens
    
    text = replaceTerm(text)
    # lemmatize text
    text = " ".join([t.lemma_ for t in nlp(text, disable=['tagger', 'parser', 'ner','tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler',  'ner'])])
    return(text)

In [289]:
def load_correctDict(ws):
    #it creates a dictionary out of a dataset that containes pairs of (original term, corrected term)    
    dataset = Dataset.get_by_name(ws, name='correct_Tokens')    
    corDict = dict(dataset.to_pandas_dataframe().to_dict("split")['data'])
    return corDict

In [290]:
def correct(x,corDict):
    #uses the dictionary to correct the terms
    if x in corDict.keys():
        y = corDict[x]
    else:
        y = x
    return y    

In [291]:
def get_ngrams(idf,mindf,minngram,maxngram):
    #this function returns the bi-grams and tri-grams
    tfidf = TfidfVectorizer(min_df = mindf,ngram_range = (minngram,maxngram))
    tfidf_result = tfidf.fit_transform(idf['tokenized']).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = [str(x) for x in tfidf_df.columns]
    df_i = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()
    df_i = df_i[df_i['value']>0]
    return df_i

In [292]:
def cleanComments(df,sw):
    #applies the clean text function to all texts contained in the dataset
    df = df[['CON_ROW_ID','CON_COMMENTS']]
    df['tokenized'] = df['CON_COMMENTS'].apply(clean_text,args = (sw))
    df = df.fillna('N/A')
    df['variable'] = df['tokenized'].str.split()
    return df

In [293]:
def getTokens(df,sw):
    #The variable columns is a list. The explode method "unpivots this list"
    df = cleanComments(df,sw)
    df_f = df.explode('variable')[['CON_ROW_ID','variable']]
    return df_f

In [294]:
def getTokencount(df_f,minCount):
    #calculate the number of occurances (counts) of each token
    #tokens with count less than mincount are set to blank
    tokenCount = df_f['variable'].value_counts().to_dict()
    
    df_f['value'] = df_f['variable'].map(tokenCount)
   
    df_f.loc[(df_f['value']<minCount), 'variable'] = ' ' #addition to return even empty tokens
    
    return df_f

In [295]:
def performNLP(workspace,minCount,mindf,minngram,maxngram,deleteEmptyTokens,df):
    sw = loadStopWords(workspace)
    
    df = cleanComments(df,sw)
    
    df_f = getTokens(df,sw)

    df_f.count()
    
    df_f = df_f.fillna(' ')
    
    df_f = getTokencount(df_f,minCount)
    
    #try:
    #    df_f = df_f.append(get_ngrams(df,mindf,minngram,maxngram ))
    #except:
    #    print('no bigramms or trigramms were added')
    
    #corDict = load_correctDict(workspace)     
    
    #df_f['token'] = df_f['variable'].apply(lambda x : correct(x,corDict))
    
    #df_f.loc[(df_f['token'].str.len() <2), 'token'] = ' ' #single character tokens are set to blank
    
    df_f['token'] = df_f['variable']
    
    df_f = df_f.sort_values(['CON_ROW_ID','token'])
    
    if deleteEmptyTokens:
        df_f = df_f[df_f['token'] != ' ']
    
    df_f = df_f[['CON_ROW_ID','token']].drop_duplicates()
    
    return df_f

In [296]:
def loadTexts(workspace,datasetName):
    #loads the texts to be analyzed
    dataset = Dataset.get_by_name(workspace, name=datasetName)
    df = dataset.to_pandas_dataframe()
    df= df[['CON_ROW_ID','CON_COMMENTS']]
    return df   

In [297]:
def exportResults(workspace,datastore,fileName,df_f):
    df_f.to_csv(fileName+'.txt',sep =',',line_terminator='\r\n',index = False)
    fil = [os.getcwd()+'/'+ fileName+'.txt']
    datastore.upload_files(fil, target_path='UI/NLP', overwrite=True, show_progress=True)
    
    

<b>The commended-out code is for debuging purposes</b>

In [298]:
#txt = 'H eurobank είναι καλύτερη τράπεζα στον κόσμο'
#com = {'CON_ROW_ID':[1],'CON_COMMENTS':[txt]}
#df = pd.DataFrame(com)

In [299]:
df = loadTexts(workspace,datasetName)

In [300]:
#df = df.head(10000)

In [301]:
df.head(1000)

,CON_ROW_ID,CON_COMMENTS
0,2673060,19662205 ΤΑΜΑΝΑΚΗ ΧΡΥΣΟΥΛΑ &
1,6182214,1966346 &
2,6053283,1966346 &
3,6573619,19664841 ΔΑΣΚΑΛΑΚΗ ΕΥΑΓ.ΜΑΡΚΟΣ&
4,6573744,19664841 ΔΑΣΚΑΛΑΚΗ ΕΥΑΓ.ΜΑΡΚΟΣ&
...,...,...
995,1576787,1o GYMNASIO RAFINAS 256/06-11-&
996,1170116,1o GYMNASIO RAFINAS 256/06-11-&
997,4538035,1o GYMNASIO RAFINAS 531/31-10-&
998,3926532,1o GYMNASIO RAFINAS 531/31-10-&


In [302]:
df_f = performNLP(workspace, minCount, mindf, minngram, maxngram, deleteEmptyTokens, df)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [303]:
df_f.head(1000)

,CON_ROW_ID,token
67850,5488,2o
67850,5488,noembrioy
8113,5505,5
8113,5505,τηλεφωνο
8116,5558,6
...,...,...
58760,261329,αε
2568,261674,2
89334,262104,arisagor
89334,262104,stoiximangr


In [304]:
exportResults(workspace,datastore,fileName,df_f)

#run.complete()

Uploading an estimated of 1 files
Uploading /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/vkontogcompute/code/Users/mchouliarea/Txns_NLP_202011_part2_exp.txt
Uploaded /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/vkontogcompute/code/Users/mchouliarea/Txns_NLP_202011_part2_exp.txt, 1 files out of an estimated total of 1
Uploaded 1 files


In [321]:
pd.merge(df_f,df, how="inner",on=['CON_ROW_ID']).head(10000000).to_excel('NLP{0}.xlsx'.format(fileName))

In [322]:
#df_f['token'].value_counts()

In [323]:
#df[df['CON_ROW_ID']==5355548]

In [324]:
#df_f[df_f['token']=='αριθμοςκυκλοφοριας']